## DB연결

In [2]:
from pymongo import MongoClient
# 네트워크 연결 규칙 : protocol://ip:port/path 
client = MongoClient('mongodb://192.168.0.50:27017/')

db = client["DB_SGMN"]
collection = db["COL_SCRAPPING_TOSS_COMMENT_HISTORY"]

In [3]:
getDatafromDB = collection.find()


data = list(getDatafromDB)


In [4]:
import pandas as pd

In [5]:
dfData = pd.DataFrame(data)

In [ ]:
dfData.info()

In [ ]:
dfDataEdited = dfData.drop(labels='_id',axis=1)
dfDataEdited

In [8]:
dataToDicOnlyCommentAndDate = dfDataEdited[dfDataEdited['SYMBOL'] == 'TSLA'][['COMMENT', 'DATE', 'SYMBOL']].to_dict(orient="list")


In [ ]:
commentDatasList = dfDataEdited.groupby('DATE')['COMMENT'].apply(list).to_dict()
commentDatasList
type(commentDatasList)

In [ ]:
dateList = dataToDicOnlyCommentAndDate['DATE']
commentList = dataToDicOnlyCommentAndDate['COMMENT']
# dateList
type(dateList)

In [ ]:
commentList
len(commentList)

## TF-IDF 사용 문장 유사도 작성

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer # Word Embedding
# tfidfVectorizer = TfidfVectorizer()
tfidfVectorizer = TfidfVectorizer()
tfidfVectorizer.fit(commentList) # 학습용
result_vectors = tfidfVectorizer.transform(commentList) # 서비스용


In [ ]:
result_vectors.toarray().shape

In [ ]:
result_vectors.toarray().shape, tfidfVectorizer.get_feature_names_out().shape

In [15]:
vectors_100 = result_vectors[:100]

In [ ]:
## 문장간 유사도 측정

from sklearn.metrics.pairwise import cosine_similarity
df_similarity = cosine_similarity(vectors_100, vectors_100)
df_similarity

In [ ]:
import pandas as pd

pd.DataFrame(data=df_similarity

, index=commentList[:100]
, columns=commentList[:100]
)

In [ ]:
# new_sentence = '가상현실(VR)과 증강현실(AR) 기술은 엔터테인먼트, 교육, 의료 등 다양한 분야에서 새로운 경험을 제공한다.'
new_sentence = '테슬라는 신이야'

new_tfidfVectorization = tfidfVectorizer.transform([new_sentence]) # Vocabulary 기준 Imbedding 전환
new_tfidfVectorization

In [ ]:
similarity_array = cosine_similarity(new_tfidfVectorization, result_vectors)
argmax_similarity = similarity_array.argmax()
commentList[argmax_similarity]